# notebooks.GNN_for_maxcut

In [145]:
#!pip install torch-geometric

In [146]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
import numpy as np

In [147]:
def generate_dataset(num_samples=1000, graph_size=4, seed=None):
    if seed is not None:
        np.random.seed(seed)
    datasets = []
    labels = []
    for _ in range(num_samples):
        q = np.random.randint(-5, 6, (graph_size, graph_size))
        q_s = (q + q.T) / 2
        spins = np.random.choice([-1, 1], size=graph_size)
        cut_value = 0
        for i in range(graph_size):
            for j in range(i + 1, graph_size):
                if spins[i] != spins[j]:
                    cut_value += abs(q_s[i, j])
        input_features = q_s[np.triu_indices(graph_size, k=1)]
        datasets.append(input_features)
        labels.append(cut_value)
    return torch.tensor(datasets, dtype=torch.float32), torch.tensor(labels, dtype=torch.float32)

In [148]:
def create_single_graph(q_s):
    graph_size = q_s.shape[0]
    edge_index = torch.combinations(torch.arange(graph_size), r=2).t()
    edge_weights = q_s[np.triu_indices(graph_size, k=1)]
    data = Data(
        x=torch.eye(graph_size),
        edge_index=edge_index,
        edge_attr=torch.tensor(edge_weights, dtype=torch.float32).unsqueeze(1),
        y=None,
    )
    return data

In [149]:
class GNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = torch.mean(x, dim=0)
        x = self.fc(x)
        return x

In [150]:
def prepare_dataset(num_samples=1000, graph_size=4):
    datasets = []
    labels = []
    features, target_labels = generate_dataset(num_samples=num_samples, graph_size=graph_size)
    for i in range(num_samples):
        edge_index = torch.combinations(torch.arange(graph_size), r=2).t()
        data = Data(
            x=torch.eye(graph_size),
            edge_index=edge_index,
            edge_attr=features[i].unsqueeze(1),
            y=target_labels[i],
        )
        datasets.append(data)
    return datasets

In [151]:
def train_gnn(num_samples=1000, graph_size=4, epochs=50, batch_size=32):
    dataset = prepare_dataset(num_samples=num_samples, graph_size=graph_size)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    model = GNN(input_dim=graph_size, hidden_dim=16, output_dim=1)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    loss_fn = torch.nn.MSELoss()
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()
            output = model(batch)
            loss = loss_fn(output.squeeze(), batch.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")
    return model

In [152]:
def test_gnn(model, num_samples=100, graph_size=4):
    test_dataset = prepare_dataset(num_samples=num_samples, graph_size=graph_size)
    dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            output = model(batch)
            loss = F.mse_loss(output.squeeze(), batch.y)
            total_loss += loss.item()
    print(f"Test Loss: {total_loss / len(dataloader):.4f}")

In [153]:
def test_single_graph(model, graph):
    model.eval()
    with torch.no_grad():
        prediction = model(graph).item()
    return prediction

In [154]:
def exact_maxcut(q_s):
    n = q_s.shape[0]
    max_cut = 0
    for partition in range(1, 1 << n - 1):
        set_a = [i for i in range(n) if (partition & (1 << i))]
        set_b = [i for i in range(n) if not (partition & (1 << i))]
        cut_value = 0
        for i in set_a:
            for j in set_b:
                cut_value += abs(q_s[i, j])

        max_cut = max(max_cut, cut_value)
    return max_cut

In [155]:
q_s = np.array([[0, 3, 0, -2],
                [3, 0, 4, 0],
                [0, 4, 0, 1],
                [-2, 0, 1, 0]])

test_graph = create_single_graph(q_s)

In [156]:
model = train_gnn(num_samples=1000, graph_size=4, epochs=100)
true_maxcut = exact_maxcut(q_s)
predicted_cut = test_single_graph(model, test_graph)
approximation_ratio = predicted_cut / true_maxcut if true_maxcut != 0 else 0
print(f"Predicted MaxCut value for the square graph: {predicted_cut:.2f}")
print(f"True MaxCut value for the square graph: {true_maxcut:.2f}")
print(f"Approximation Ratio: {approximation_ratio:.2f}")

Epoch 1/100, Loss: 827.2873
Epoch 2/100, Loss: 330.7068
Epoch 3/100, Loss: 321.5484
Epoch 4/100, Loss: 315.8487
Epoch 5/100, Loss: 318.1853
Epoch 6/100, Loss: 322.8162
Epoch 7/100, Loss: 316.2698
Epoch 8/100, Loss: 323.6266
Epoch 9/100, Loss: 325.0360
Epoch 10/100, Loss: 320.6282
Epoch 11/100, Loss: 320.4912
Epoch 12/100, Loss: 319.1008
Epoch 13/100, Loss: 323.4767
Epoch 14/100, Loss: 319.1054
Epoch 15/100, Loss: 321.8447
Epoch 16/100, Loss: 327.5805
Epoch 17/100, Loss: 321.0409
Epoch 18/100, Loss: 324.4548
Epoch 19/100, Loss: 318.8719
Epoch 20/100, Loss: 329.6692
Epoch 21/100, Loss: 322.8546
Epoch 22/100, Loss: 316.7159
Epoch 23/100, Loss: 323.1354
Epoch 24/100, Loss: 320.2368
Epoch 25/100, Loss: 325.6627
Epoch 26/100, Loss: 317.2012
Epoch 27/100, Loss: 315.7739
Epoch 28/100, Loss: 320.9980
Epoch 29/100, Loss: 328.7326
Epoch 30/100, Loss: 325.3058
Epoch 31/100, Loss: 324.5121
Epoch 32/100, Loss: 324.7326
Epoch 33/100, Loss: 321.4706
Epoch 34/100, Loss: 321.5275
Epoch 35/100, Loss: 317